# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending a question to it and getting it scored for matches in the original questions.

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [1]:
import json
from utilities import write_json_to_file

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [2]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

docker_login = '<YOUR_DOCKER_LOGIN>'
image_name = docker_login + '/mlaksdep' # 'fboylu/mlaksdep' Use this image if you want to skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select the subscription you want to use 

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [3]:
!az login -o table

In [4]:
!az account set --subscription $selected_subscription

In [5]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [7]:
!az provider register -n Microsoft.ContainerService

In [8]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [3]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbakscpurg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster  with 5 nodes in the resource group we created earlier.

In [6]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 5,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_D4_v2",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbakscpurg-edf507",
  "fqdn": "fbaksclust-fbakscpurg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhL

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [6]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [7]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustercpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [8]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     53d       v1.9.6
aks-nodepool1-13347348-1   Ready     agent     53d       v1.9.6
aks-nodepool1-13347348-2   Ready     agent     53d       v1.9.6
aks-nodepool1-13347348-3   Ready     agent     53d       v1.9.6
aks-nodepool1-13347348-4   Ready     agent     53d       v1.9.6


Let's check the pods on our cluster.

In [6]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          53d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          53d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          53d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          53d
kube-system   kube-proxy-225hz                        1/1       Running   0          53d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          53d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          53d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          53d
kube-system   kube-proxy-tvnpv                        1/1       Running   0          53d
kube-system   kube-svc-redirect-278jk                 1/1       Running   3          53d
kube-system   kube-sv

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the image name and cpu requests and limits for pods. We first start with  deploying 2 pods.

In [7]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-ml"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-ml",
                      "image": image_name,

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-ml"
      }
   }
}

In [8]:
write_json_to_file(app_template, 'az-ml.json')

In [9]:
write_json_to_file(service_temp, 'az-ml.json', mode='a')

Let's check the manifest created.

In [10]:
!cat az-ml.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-ml"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-ml"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/mlaksdep",
                        "name": "azure-ml",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
              

Next, we will use kubectl create command to deploy our application.

In [11]:
!kubectl create -f az-ml.json

deployment.apps/azure-ml created
service/azure-ml created


Let's check if the pod is deployed.

In [9]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d9b465b7f-7ssmm               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-hgwcc               1/1       Running   0          1h
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          53d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          53d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          53d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          53d
kube-system   kube-proxy-225hz                        1/1       Running   0          53d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          53d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          53d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          53d
kube-system   kube-prox

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [20]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                  SOURCE                              MESSAGE
1m          1m           1         azure-ml-7d9b465b7f-24mz5.1546d1cb17d5f1fd   Pod                                      Normal    Scheduled               default-scheduler                   Successfully assigned azure-ml-7d9b465b7f-24mz5 to aks-nodepool1-13347348-0
1m          1m           1         azure-ml-7d9b465b7f-24mz5.1546d1cb97603a3f   Pod                                      Normal    SuccessfulMountVolume   kubelet, aks-nodepool1-13347348-0   MountVolume.SetUp succeeded for volume "default-token-lls54" 
1m          1m           1         azure-ml-7d9b465b7f-24mz5.1546d1cbdc5b85be   Pod          spec.containers{azure-ml}   Normal    Pulling                 kubelet, aks-nodepool1-13347348-0   pulling image "fboylu/mlaksdep"
1m          1m           1         azure-ml-7d9b465b7f-24s

1m          1m           1         azure-ml-7d9b465b7f-tdpcx.1546d1cda2adbf1d   Pod          spec.containers{azure-ml}   Normal    Pulled                  kubelet, aks-nodepool1-13347348-4   Successfully pulled image "fboylu/mlaksdep"
1m          1m           1         azure-ml-7d9b465b7f-tdpcx.1546d1cdb48d42a2   Pod          spec.containers{azure-ml}   Normal    Created                 kubelet, aks-nodepool1-13347348-4   Created container
1m          1m           1         azure-ml-7d9b465b7f-tdpcx.1546d1cdbe141734   Pod          spec.containers{azure-ml}   Normal    Started                 kubelet, aks-nodepool1-13347348-4   Started container
1m          1m           1         azure-ml-7d9b465b7f-tf6s7.1546d1cb173d30ca   Pod                                      Normal    Scheduled               default-scheduler                   Successfully assigned azure-ml-7d9b465b7f-tf6s7 to aks-nodepool1-13347348-1
1m          1m           1         azure-ml-7d9b465b7f-tf6s7.1546d1cb9b9547a3   

Check the logs for the first application pod.

In [13]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [14]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-08-01 15:38:48,728 CRIT Supervisor running as root (no user in config file)
2018-08-01 15:38:48,730 INFO supervisord started with pid 1
2018-08-01 15:38:49,733 INFO spawned: 'program_exit' with pid 10
2018-08-01 15:38:49,735 INFO spawned: 'nginx' with pid 11
2018-08-01 15:38:49,737 INFO spawned: 'gunicorn' with pid 12
2018-08-01 15:38:50,778 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
{"stack_info": null, "path": "/code/driver.py", "level": "INFO", "tags": [], "message": "Model object loading time: 668.3 ms", "timestamp": "2018-08-01T15:38:51.443240Z", "host": "azure-ml-7d9b465b7f-7ssmm", "logger": "model_driver"}
Initialising
{"stack_info": null, "path": "/opt/conda/envs/py3.5/lib/python3.5/site-packages/werkzeug/_internal.py", "level": "INFO", "tags": [], "message": " * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)", "timestamp": "2018-08-01T15:38:51.449546Z", "msg": " * Running on %s://%s:%d/ %s", "host": "

In [15]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   2         2         2            2           1h


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [16]:
!kubectl get service azure-ml

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
azure-ml   LoadBalancer   10.0.126.171   137.117.41.76   80:30675/TCP   1h


# Scaling

In this part, we scale the number of pods to make sure we fully utilize the AKS cluster.

In [17]:
!kubectl scale --current-replicas=2 --replicas=35 deployment/azure-ml

deployment.extensions/azure-ml scaled


In [21]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d9b465b7f-24mz5               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-24s7h               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-27bq6               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-2kwqw               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-5cj9x               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-6cl8r               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-6dmf6               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-7hqb6               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-7rhf7               1/1       Running   0          2m
default       azure-ml-7d9b465b7f-7ssmm               1/1       Running   0          1h
default       azure-ml-7d9b465b

In [22]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   35        35        35           35          1h


Next, we will [test our web application deployed on AKS](06_Test_WebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [48]:
!kubectl delete -f az-ml.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

In [4]:
!az group delete --name $resource_group -y

 - Finished ..